# Initiailize VW executor

In [6]:
from vw_executor.vw import Vw
from vw_executor.handlers import MLFlowHandler

vw = Vw('.vw_cache2', handlers = [MLFlowHandler("vw_experiment_21")])

# Get the data

In [7]:
inputs = [
    'vw_executor/tests/data/cb_1000_0.json',
    'vw_executor/tests/data/cb_1000_1.json', 
    ]
inputs

['vw_executor/tests/data/cb_1000_0.json',
 'vw_executor/tests/data/cb_1000_1.json']

# Simple vw run

In [ ]:
result = vw.train(inputs, '--cb_explore_adf --dsjson')
result[1].stdout.raw

# Options that you probably need for training
--preserve_performance_counters - make all counters preserved between different files <br/>
--save_resume - model that is saved as output after each file processing, has all information required for further learning (not just regressor weights) </br>
-P $some_number    - make progressive loss report every $some_number examples

In [ ]:
result = vw.train(inputs, '--cb_explore_adf --dsjson --preserve_performance_counters --save_resume -P 10')
result[1].stdout.raw

# Define options grid and train

In [8]:
from vw_executor.vw_opts import Grid
import pandas as pd

opts = pd.DataFrame(Grid({
    '#base': ['--cb_explore_adf -P 10 --preserve_performance_counters --save_resume --dsjson'],
    '--cb_type': ['ips', 'mtr'],
    '--learning_rate': [0.001, 0.01, 0.1]}))
opts

,#base,--cb_type,--learning_rate
0,--cb_explore_adf -P 10 --preserve_performance_...,mtr,0.100
1,--cb_explore_adf -P 10 --preserve_performance_...,ips,0.100
2,--cb_explore_adf -P 10 --preserve_performance_...,mtr,0.001
3,--cb_explore_adf -P 10 --preserve_performance_...,mtr,0.010
4,--cb_explore_adf -P 10 --preserve_performance_...,ips,0.010
5,--cb_explore_adf -P 10 --preserve_performance_...,ips,0.001


In [9]:
result = vw.train(inputs, opts)
result

AttributeError: 'NoneType' object has no attribute 'items'

## Get access to losses and job artifacts

In [ ]:
print(f'loss[0]: {result[0].loss}')
print(f'outputs: {result[0].outputs}')

# Pandas wrappers

In [ ]:
opts = pd.DataFrame(Grid({
    '#base': ['--ccb_explore_adf -P 10 --preserve_performance_counters --save_resume'],
    '--cb_type': ['ips', 'mtr'],
    '#format': ['', '--dsjson']}))
opts

In [ ]:
result = vw.train(inputs, opts)
result

## Investigate failure

In [ ]:
result[result['!Status'] == 'Failed'].iloc[0]['!Job'].failed.stdout.raw

## Explore metrics 

In [ ]:
result[result['!Status'] == 'Success'].iloc[0]['!Job'].loss_table

In [ ]:
import matplotlib.pyplot as plt
[(-r['!Job'].loss_table['loss']).plot(label=f"{r['!Job'].name}", figsize=(10,6)) for i, r in result[result['!Status'] == 'Success'].iterrows()]
plt.legend()

# Interactive mode

In [ ]:
%matplotlib widget 
from vw_executor.vw_opts import InteractiveGrid as Grid
from ipywidgets import FloatLogSlider, FloatSlider
opts = Grid({
    '#base': ['--cb_explore_adf -P 10 --preserve_performance_counters --save_resume --dsjson'],
    '--cb_type': ['ips', 'mtr'],
    '--learning_rate': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1]
#    '--learning_rate': FloatLogSlider(min=-10, max=10, base=10, step=1)
    })
vw.train(inputs, opts)

# Get more logging

## Log to console

In [ ]:
from vw_executor.vw import Vw
from vw_executor.loggers import ConsoleLogger

vw = Vw('.vw_cache',
    loggers = [ConsoleLogger('DEBUG')])

vw.train(inputs, '--cb_explore_adf --dsjson')

## Log to file

In [ ]:
from vw_executor.vw import Vw
from vw_executor.loggers import FileLogger

vw = Vw('.vw_cache',
    loggers = [FileLogger('log.txt', 'DEBUG')])

vw.train(inputs, '--cb_explore_adf --dsjson')

## Log to multiple files

In [ ]:
from vw_executor.vw import Vw
from vw_executor.loggers import MultiFileLogger

vw = Vw('.vw_cache',
    loggers = [MultiFileLogger('artifacts', 'DEBUG')])

vw.train(inputs, ['--cb_explore_adf --dsjson --epsilon 0.2', '--cb_explore_adf --dsjson --epsilon 0.3'])

## Publish artifacts

In [ ]:
from vw_executor.vw import Vw
from vw_executor.handlers import ArtifactCopy

vw = Vw('.vw_cache',
    handlers = [ArtifactCopy('artifacts', outputs=['-p'])])

vw.train(inputs, ['--cb_explore_adf --dsjson --epsilon 0.2', '--cb_explore_adf --dsjson --epsilon 0.3'], ['-p'])